In [19]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pymongo
import jieba
import jieba.analyse
# import synonyms
import time

In [37]:
URL = 'https://wh.meituan.com/meishi/'
ops = webdriver.ChromeOptions()
ops.add_argument('--headless')
ops.add_argument('--no-sandbox')
driver = webdriver.Chrome(chrome_options=ops)
# driver.implicitly_wait(5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [126]:
client = pymongo.MongoClient("mongodb://101.132.171.223:27017/")
db = client['shoprec']

In [121]:
# Utils
def getTypes():
    driver = webdriver.Chrome(chrome_options=ops)
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    body = soup.body
    _types = body.find('div', attrs={'data-reactid': '17'})
    _a = _types.contents[2].find_all('a')
    types = list(filter(typeFilter, (map(lambda x : {'name': x.text, 'index': x['href'].split('/')[-2]}, _a))))
    return types
    
def typeFilter(x):
    return x['name'] != '代金券' and x['name'] != '其他美食'

def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
            
def writeList(content, filename):
    f = open(filename, 'w', encoding='utf8')
    f.truncate()
    contents_to_write = list(map(lambda x : x + '\n', content))
    contents_to_write[-1] = contents_to_write[-1].strip('\n')
    f.writelines(contents_to_write)
    f.close()
    
def readList(filename):
    f = open(filename, 'r', encoding='utf8')
    contents = list(map(lambda x: x.strip(), list(f)))
    return contents

In [122]:
_types = getTypes()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [127]:
col = db['type']
col.insert_many(_types)

In [128]:
x = ['1', '2', '3']